In [35]:
Primermpy as np
from obspy import read, UTCDateTime

def leer_archivos_reftek(ruta_archivo, ruta_salida):
    """
    Lee y procesa archivos Reftek utilizando la librería ObsPy.
    
    Args:
        ruta_archivo (str): Ruta del archivo Reftek a procesar.
        ruta_salida (str): Ruta de salida para los archivos procesados.
        
    Returns:
        stream (obspy.Stream): Stream de datos procesados.
    """
    stream = read(ruta_archivo, format='REFTEK')
    stream.write(ruta_salida, format='MSEED')
    return stream

def proc_cru(*args):
    """
    Procesamiento de los datos 'crudos'
    Inputs:
        args[0] - Nombre del programa para el procesamiento
        args[1] - ID estacione(s) Reftek
        args[2] - Fecha de evento (aaaaddd)
        args[3] - Canal(es)
        args[4] - Archivo(s) a procesar
        args[5] - Directorio de archivos de entrada
        args[6] - Directorio de los programas
        args[7] - Directorio de salida
        args[8] - Usar programas reftek (bool)
    Estructura:
        proc_cru(METODO, ESTACION(ES), FECHA(S), CANAL(ES), ARCHIVO(S), 
                 DIRECTORIO DE ARCHIVOS EN CRUDO, DIRECTORIO DE PROGRAMAS, 
                 CARPETA DE SALIDA ARCHIVOS)
    Ejemplo de uso:
        proc_cru('pas2asc', 'D884', '2021044', '1', '050000000_0036EE80', [], [], 'C:\\Users\\Usuario\\Desktop\\DATA_ESTACION')

        proc_cru('pas2asc', ['D884', 'D8A4'], ['2021044', '2021049'], ['1', '2'], ['050000000_0036EE80', '060000000_0036EE80'], [], [], 'C:\\Users\\Usuario\\Desktop\\DATA_ESTACION')

        proc_cru('pas2asc', ['D884', 'D8A4'], ['2021044', '2021049'], ['1', '2'], ['050000000_0036EE80', '060000000_0036EE80'], [], [], 'C:\\Users\\Usuario\\Desktop\\DATA_ESTACION')

        archivos en directorio de salida por defecto
        proc_cru('pas2asc', 'D884', '2021044', '1', [], 'C:\\reftek\\data', 'C:\\reftek\\bin')

        Se Puede usar solo con las dos primeras partes
        proc_cru('pas2asc', 'D884') # procesará todos los datos existentes
    
    ***Agregar discretizacion con la metadata de los archivos mseed***
    _________________________________________________________________________
    """

    # Verificar número de argumentos
    if len(args) < 2 or len(args) > 9:
        raise ValueError("Número incorrecto de argumentos. Se esperan entre 2 y 9 argumentos.")

    programa = args[0]
    codigo_estacion = args[1]

    if len(args) > 2:
        if len(args) == 3:
            fecha = args[2]
            canal = []
            archivo = []
        elif len(args) == 4:
            fecha = args[2]
            canal = args[3]
            archivo = []
        elif len(args) == 5 or len(args) > 5:
            fecha = args[2]
            canal = args[3]
            archivo = args[4]
    else:
        fecha = []
        canal = []
        archivo = []
        
    # Verificamos si fecha no es una lista y tiene solo un elemento
    if not isinstance(fecha, list) and len(fecha) == 1:
        if len(fecha) % 7 == 0:
            fecha = [fecha[i:i+7] for i in range(0, len(fecha), 7)]
            try:
                UTCDateTime(fecha[0], format='%Y%j')
            except ValueError:
                raise ValueError('Formato de fecha incorrecto')
        elif len(fecha) % 10 == 0:
            fecha = [fecha[i:i+10] for i in range(0, len(fecha), 10)]
            try:
                UTCDateTime(fecha[0], format='%d-%m-%Y')
            except ValueError:
                raise ValueError('Formato de fecha incorrecto')
            fecha = [UTCDateTime(f, format='%d-%m-%Y').strftime('%Y%j') for f in fecha]
        else:
            raise ValueError('Formato de fecha incorrecto')
    elif len(fecha) > 1 and not isinstance(fecha, list):
        fecha = list(fecha)
        if len(fecha[0]) == 7:
            try:
                UTCDateTime(fecha[0], format='%Y%j')
            except ValueError:
                raise ValueError('Formato de fecha incorrecto')
        elif len(fecha[0]) == 10:
            try:
                UTCDateTime(fecha[0], format='%d-%m-%Y')
            except ValueError:
                raise ValueError('Formato de fecha incorrecto')
            fecha = [UTCDateTime(f, format='%d-%m-%Y').strftime('%Y%j') for f in fecha]
    elif isinstance(fecha, list):
        if len(fecha[0]) == 10:
            fecha = [UTCDateTime(f, format='%d-%m-%Y').strftime('%Y%j') for f in fecha]

    if not isinstance(archivo, list) and len(archivo) == 1:
        if len(archivo) % 18 == 0:
            archivo = [archivo[i:i+18] for i in range(0, len(archivo), 18)]
        else:
            raise ValueError('Nombre del archivo(s) incorrecto')
    elif len(archivo) > 1 and not isinstance(archivo, list):
        archivo = list(archivo)
        
    if not isinstance(codigo_estacion, list) and len(codigo_estacion) == 1:
        if len(codigo_estacion) % 4 == 0:
            codigo_estacion = [codigo_estacion[i:i+4] for i in range(0, len(codigo_estacion), 4)]
        else:
            raise ValueError('Nombre del codigo_estacion(s) incorrecto')
    elif len(codigo_estacion) > 1 and not isinstance(codigo_estacion, list):
        codigo_estacion = list(codigo_estacion)

    if not isinstance(canal, str) and canal:
        canal = [str(item) for item in canal]
    elif isinstance(canal, str) and canal:
        canal = [canal[i] for i in range(max(len(canal), len(canal[0])))]

    if len(args) > 5 and isinstance(args[5], str):
        dir_arc = args[5]
        if not os.path.isdir(dir_arc):
            raise ValueError('Error en el ingreso del directorio')
    elif len(args) < 6 or not args[5]:
        dir_arc = 'C:\Practicas\datos'
        if not os.path.isdir(dir_arc):
            raise ValueError('No existe el directorio de los datos')
    else:
        raise ValueError('Error en el ingreso del directorio')

    dir_data = [d for d in os.listdir(dir_arc) if os.path.isdir(os.path.join(dir_arc, d)) and d not in ['.', '..']]
    if fecha:
        dir_data = [d for d in dir_data if d in fecha]

    if len(args) > 7 and isinstance(args[7], str):
        dir_archivos_salida = os.path.join(args[7], 'DATOS_PROCESADOS')
        if not os.path.isdir(dir_archivos_salida):
            os.makedirs(dir_archivos_salida)
    elif len(args) < 8 or not args[7]:
        dir_salida_por_defecto = os.listdir('.\\')[0]
        dir_archivos_salida = os.path.join(dir_salida_por_defecto, 'OUTPUT', 'DATOS_PROCESADOS')

    estaciones_dr_c = [None] * len(dir_data)

    # Inicialización de dr_c como una lista tridimensional
    dr_c = [[[[] for _ in range(len(dir_data))] for _ in range(len(codigo_estacion))] for _ in range(len(canal))]

    # elementos de discretización de los archivos
    for h, dir_item in enumerate(dir_data):
        dir_in = [d for d in os.listdir(dir_item.path) if os.path.isdir(os.path.join(dir_item.path, d)) and d not in ['.', '..']]
        # nombre de los directorios
        if codigo_estacion:
            dir_in = [d for d in dir_in if d in codigo_estacion]
        estaciones_dr_c[h] = dir_in
        
        for l, dir_in_item in enumerate(dir_in):
            dir_canal = [d for d in os.listdir(os.path.join(dir_item.path, dir_in_item)) if os.path.isdir(os.path.join(dir_item.path, dir_in_item, d)) and d not in ['.', '..']]
            if canal and any(c in dir_canal for c in canal):
                dir_canal = [d for d in dir_canal if d in canal]
            elif canal and any(c not in dir_canal for c in canal):
                raise ValueError('El canal introducido no es correcto')
            
            dir_salida = os.path.join(dir_archivos_salida, dir_in_item)
            dir_salida_temp = os.path.join(dir_salida, datetime.datetime.strptime(dir_item.name, '%Y%j').strftime('%d-%m-%Y'))
            if not os.path.exists(dir_salida_temp):
                os.makedirs(dir_salida_temp)
            
            for k, dir_canal_item in enumerate(dir_canal):
                dir_archivos = [f for f in os.listdir(os.path.join(dir_item.path, dir_in_item, dir_canal_item)) if not os.path.isdir(os.path.join(dir_item.path, dir_in_item, dir_canal_item, f))]
                if len([d for d in os.listdir(os.path.join(dir_item.path, dir_in_item, dir_canal_item)) if os.path.isdir(os.path.join(dir_item.path, dir_in_item, dir_canal_item, d))]) == 2:
                    dir_archivos = []
                    break
                try:
                    dr_c[k][l][h] = dir_archivos
                except Exception as e:
                    print('CELDA VACIA', e)
    
    dir_dat_fch = [d.name for d in dir_data]

    if len(args) > 6 and isinstance(args[6], str):
        dir_programas = args[6]
        if (not os.path.isdir(dir_programas) or not os.path.exists(dir_programas)) and args[8]:
            raise ValueError('Error en el ingreso del directorio de programas')
    elif len(args) < 7 or not args[6]:
        dir_programas = 'C:\Practicas\programas_r\bin'
        if not os.path.exists(dir_programas):
            raise ValueError('No existe el directorio de los datos')
    else:
        raise ValueError('Error en el ingreso del directorio')
    
    dim_dr_c = (len(dr_c), len(dr_c[0]), len(dr_c[0][0]))  # Dimensiones de dr_c
    for fecha_idx in range(dim_dr_c[2]):
        for estacion_idx in range(dim_dr_c[1]):
            for canal_idx in range(dim_dr_c[0]):
                if dr_c[canal_idx][estacion_idx][fecha_idx]:
                    temp_archs = [f.name for f in dr_c[canal_idx][estacion_idx][fecha_idx]]
                    if archivo:
                        temp_archs = [arch for arch in temp_archs if arch in archivo]

                    dir_salida = os.path.join(dir_archivos_salida, estaciones_dr_c[fecha_idx][estacion_idx])
                    crp_temp = dr_c[canal_idx][estacion_idx][fecha_idx][0].folder

                    for temp_arch in temp_archs:
                        if args[8]:  # usar programa reftek
                            linea_cd_stm = os.path.join(
                                dir_programas, programa,
                                os.path.join(crp_temp, temp_arch),
                                os.path.join(dir_salida, UTCDateTime(dir_dat_fch[fecha_idx], format='%Y%j').strftime('%d-%m-%Y'))
                            )
                            subprocess.run(linea_cd_stm, shell=True)
                        else:
                            ruta_archivo = os.path.join(crp_temp, temp_arch)
                            ruta_salida = os.path.join(dir_salida, UTCDateTime(dir_dat_fch[fecha_idx], format='%Y%j').strftime('%d-%m-%Y'), temp_arch + '.mseed')
                            leer_archivos_reftek(ruta_archivo, ruta_salida)

                    if canal_idx == dim_dr_c[0] - 1:
                        print('Se han procesado los datos crudos')

    print('Ha finalizado la transformación de los archivos')
    print('\a')  # Beep en algunos sistemas

In [4]:
# Ejemplo de uso 1
proc_cru('pas2asc', 'D884', '2021044', '1', '050000000_0036EE80', [], [], 'C:\\Users\\Usuario\\Desktop\\DATA_ESTACION')

# Ejemplo de uso 2
proc_cru('pas2asc', ['D884', 'D8A4'], ['2021044', '2021049'], ['1', '2'], ['050000000_0036EE80', '060000000_0036EE80'], [], [], 'C:\\Users\\Usuario\\Desktop\\DATA_ESTACION')

# Ejemplo de uso 3
proc_cru('pas2asc', 'D884', '2021044', '1', [], 'C:\\reftek\\data', 'C:\\reftek\\bin')



ValueError: No existe el directorio de los datos

In [36]:
proc_cru('pas2asc', 'D884', '2021044', '1', [], 'C:\Practicas\datos')

ValueError: No existe el directorio de los datos

In [44]:
import os
from datetime import datetime
import subprocess

def proc_cru(*args):
    """
    Procesamiento de los datos 'crudos'
    ... (docstring original) ...
    """

    # Verificar número de argumentos
    if len(args) < 2 or len(args) > 9:
        raise ValueError("Número incorrecto de argumentos. Se esperan entre 2 y 9 argumentos.")

    programa = args[0]
    codigo_estacion = args[1]

    if len(args) > 2:
        if len(args) == 3:
            fecha = args[2]
            canal = []
            archivo = []
        elif len(args) == 4:
            fecha = args[2]
            canal = args[3]
            archivo = []
        elif len(args) == 5 or len(args) > 5:
            fecha = args[2]
            canal = args[3]
            archivo = args[4]
    else:
        fecha = []
        canal = []
        archivo = []

    # Verificar y formatear fecha
    if isinstance(fecha, str):
        fecha = [fecha]
    for i, f in enumerate(fecha):
        if len(f) == 7:
            try:
                fecha[i] = datetime.strptime(f, '%Y%j').date()
            except ValueError:
                raise ValueError('Formato de fecha incorrecto')
        elif len(f) == 10:
            try:
                fecha[i] = datetime.strptime(f, '%d-%m-%Y').date()
            except ValueError:
                raise ValueError('Formato de fecha incorrecto')
        else:
            raise ValueError('Formato de fecha incorrecto')

    # Verificar archivo
    if isinstance(archivo, str):
        archivo = [archivo]

    # Verificar codigo_estacion
    if isinstance(codigo_estacion, str):
        codigo_estacion = [codigo_estacion]

    # Verificar canal
    if isinstance(canal, str):
        canal = [canal]
    elif isinstance(canal, list):
        canal = [str(c) for c in canal]

    # Verificar directorio de entrada
    if len(args) > 5 and isinstance(args[5], str):
        dir_arc = args[5]
        if not os.path.isdir(dir_arc):
            raise ValueError('Error en el ingreso del directorio')
    elif len(args) < 6 or not args[5]:
        dir_arc = 'C:\\reftek\\data'
        if not os.path.isdir(dir_arc):
            raise ValueError('No existe el directorio de los datos')
    else:
        raise ValueError('Error en el ingreso del directorio')

    print(f"Directorio de entrada: {dir_arc}")

    dir_data = [d for d in os.listdir(dir_arc) if os.path.isdir(os.path.join(dir_arc, d)) and d not in ['.', '..']]
    if fecha:
        dir_data = [d for d in dir_data if d in [f.strftime('%Y%j') for f in fecha]]

    print(f"Directorios de datos encontrados: {dir_data}")

    # Verificar directorio de salida
    if len(args) > 7 and isinstance(args[7], str):
        dir_archivos_salida = os.path.join(args[7], 'DATOS_PROCESADOS')
        if not os.path.isdir(dir_archivos_salida):
            os.makedirs(dir_archivos_salida)
    elif len(args) < 8 or not args[7]:
        dir_salida_por_defecto = os.listdir('.\\')[0]
        dir_archivos_salida = os.path.join(dir_salida_por_defecto, 'OUTPUT', 'DATOS_PROCESADOS')

    print(f"Directorio de salida: {dir_archivos_salida}")

    estaciones_dr_c = [[] for _ in range(len(dir_data))]

    # Inicialización de dr_c como una lista tridimensional
    dr_c = [[[[] for _ in range(len(dir_data))] for _ in range(len(codigo_estacion))] for _ in range(len(canal))]

    # Obtener elementos de discretización de los archivos
    for h, dir_item in enumerate(dir_data):
        dir_in = [d for d in os.listdir(os.path.join(dir_arc, dir_item)) if os.path.isdir(os.path.join(dir_arc, dir_item, d)) and d not in ['.', '..']]
        if codigo_estacion:
            dir_in = [d for d in dir_in if d in codigo_estacion]
        estaciones_dr_c[h] = dir_in

        print(f"Estaciones encontradas en {dir_item}: {dir_in}")

        for l, dir_in_item in enumerate(dir_in):
            dir_canal = [d for d in os.listdir(os.path.join(dir_arc, dir_item, dir_in_item)) if os.path.isdir(os.path.join(dir_arc, dir_item, dir_in_item, d)) and d not in ['.', '..']]
            if canal and any(c in dir_canal for c in canal):
                dir_canal = [d for d in dir_canal if d in canal]
            elif canal and any(c not in dir_canal for c in canal):
                raise ValueError('El canal introducido no es correcto')

            print(f"Canales encontrados en {dir_in_item}: {dir_canal}")

            dir_salida = os.path.join(dir_archivos_salida, dir_in_item)
            dir_salida_temp = os.path.join(dir_salida, datetime.strptime(dir_item, '%Y%j').date().strftime('%d-%m-%Y'))
            if not os.path.exists(dir_salida_temp):
                os.makedirs(dir_salida_temp)

            for k, dir_canal_item in enumerate(dir_canal):
                dir_archivos = [f for f in os.listdir(os.path.join(dir_arc, dir_item, dir_in_item, dir_canal_item)) if not os.path.isdir(os.path.join(dir_arc, dir_item, dir_in_item, dir_canal_item, f))]
                if len([d for d in os.listdir(os.path.join(dir_arc, dir_item, dir_in_item, dir_canal_item)) if os.path.isdir(os.path.join(dir_arc, dir_item, dir_in_item, dir_canal_item, d))]) == 2:
                    dir_archivos = []
                    break
                try:
                    dr_c[k][l][h] = dir_archivos
                    print(f"Archivos encontrados en {dir_canal_item}: {dir_archivos}")
                except Exception as e:
                    print('CELDA VACIA', e)

    dir_dat_fch = [d for d in dir_data]

    if len(args) > 6 and isinstance(args[6], str):
        dir_programas = args[6]
        if (not os.path.isdir(dir_programas) or not os.path.exists(dir_programas)) and args[8]:
            raise ValueError('Error en el ingreso del directorio de programas')
    elif len(args) < 7 or not args[6]:
        dir_programas = 'C:\\reftek\\bin'
        if not os.path.exists(dir_programas):
            raise ValueError('No existe el directorio de los datos')
    else:
        raise ValueError('Error en el ingreso del directorio')

    print(f"Directorio de programas: {dir_programas}")

    dim_dr_c = (len(dr_c), len(dr_c[0]), len(dr_c[0][0]))  # Dimensiones de dr_c
    for fecha_idx in range(dim_dr_c[2]):
        for estacion_idx in range(dim_dr_c[1]):
            for canal_idx in range(dim_dr_c[0]):
                if dr_c[canal_idx][estacion_idx][fecha_idx]:
                    temp_archs = [f for f in dr_c[canal_idx][estacion_idx][fecha_idx]]
                    if archivo:
                        temp_archs = [arch for arch in temp_archs if arch in archivo]

                    print(f"Archivos a procesar en {dir_data[fecha_idx]}, {estaciones_dr_c[fecha_idx][estacion_idx]}, {canal[canal_idx]}: {temp_archs}")

                    dir_salida = os.path.join(dir_archivos_salida, estaciones_dr_c[fecha_idx][estacion_idx])
                    crp_temp = os.path.join(dir_arc, dir_data[fecha_idx], estaciones_dr_c[fecha_idx][estacion_idx], canal[canal_idx])

                    for temp_arch in temp_archs:
                        if args[8]:  # usar programa reftek
                            linea_cd_stm = os.path.join(
                                dir_programas, programa,
                                os.path.join(crp_temp, temp_arch),
                                os.path.join(dir_salida, fecha[fecha_idx].strftime('%d-%m-%Y'))
                            )
                            print(f"Ejecutando comando: {linea_cd_stm}")
                            subprocess.run(linea_cd_stm, shell=True)
                        else:
                            ruta_archivo = os.path.join(crp_temp, temp_arch)
                            ruta_salida = os.path.join(dir_salida, fecha[fecha_idx].strftime('%d-%m-%Y'), temp_arch + '.mseed')
                            print(f"Descomprimiendo: {ruta_archivo} -> {ruta_salida}")
                            descomprimir_crudos(ruta_archivo, ruta_salida) # type: ignore

                    if canal_idx == dim_dr_c[0] - 1:
                        print('Se han procesado los datos crudos')

    print('Ha finalizado la transformación de los archivos')
    print('\a')  # Beep en algunos sistemas

In [45]:
proc_cru('pas2asc', 'D884', '2021044', '1', [], 'C:\Practicas\datos', 'C:\Practicas\programas_r\bin')

Directorio de entrada: C:\Practicas\datos
Directorios de datos encontrados: []
Directorio de salida: .ipynb_checkpoints\OUTPUT\DATOS_PROCESADOS


IndexError: tuple index out of range

In [25]:
proc_cru('pas2asc', 'D884')

CELDA VACIA list index out of range


ValueError: No existe el directorio de los datos

In [104]:
from datetime import datetime
import os
import subprocess
import numpy as np

def proc_cru(*args):
    """
    Procesamiento de los datos 'crudos'
    Inputs:
        args[0] - Nombre del programa para el procesamiento
        args[1] - ID estacione(s) Reftek
        args[2] - Fecha de evento (aaaaddd)
        args[3] - Canal(es)
        args[4] - Archivo(s) a procesar
        args[5] - Directorio de archivos de entrada
        args[6] - Directorio de los programas
        args[7] - Directorio de salida
        args[8] - Usar programas reftek(bool)
    Estructura:
        proc_cru(METODO, ESTACION(ES), FECHA(S), CANAL(ES), ARCHIVO(S), DIRECTORIO DE ARCHIVOS EN CRUDO, DIRECTORIO DE PROGRAMAS, CARPETA DE SALIDA ARCHIVOS)
    Ejemplo de uso:
        proc_cru('pas2asc', 'D884', '2021044', '1', '050000000_0036EE80', [], [], 'C:/Users/Usuario/Desktop/DATA_ESTACION')
        proc_cru('pas2asc', ['D884', 'D8A4'], ['2021044', '2021049'], ['1', '2'], ['050000000_0036EE80', '060000000_0036EE80'], [], [], 'C:/Users/Usuario/Desktop/DATA_ESTACION')
        proc_cru('pas2asc', ['D884', 'D8A4'], ['2021044', '2021049'], ['1', '2'], ['050000000_0036EE80', '060000000_0036EE80'], [], [], 'C:/Users/Usuario/Desktop/DATA_ESTACION')
        archivos en directorio de salida por defecto
        proc_cru('pas2asc', 'D884', '2021044', '1', [], 'C:/reftek/data', 'C:/reftek/bin')
        Se Puede usar solo con las dos primeras partes
        proc_cru('pas2asc', 'D884') % procesara todos los datos existentes
    """

    if len(args) < 2 or len(args) > 9:
        raise ValueError("Número incorrecto de argumentos")

    programa = args[0]
    codigo_estacion = args[1]

    if len(args) > 2:
        if len(args) == 3:
            fecha = args[2]
            canal = []
            archivo = []
        elif len(args) == 4:
            fecha = args[2]
            canal = args[3]
            archivo = []
        elif len(args) >= 5:
            fecha = args[2]
            canal = args[3]
            archivo = args[4]
    else:
        fecha = []
        canal = []
        archivo = []

    if not isinstance(fecha, list) and fecha:
        try:
            fecha = [datetime.strptime(fecha, '%Y%j').strftime('%Y%j')]
        except ValueError:
            try:
                fecha = [datetime.strptime(fecha, '%d-%m-%Y').strftime('%Y%j')]
            except ValueError:
                raise ValueError('Formato de fecha incorrecto')
    elif isinstance(fecha, list):
        fecha_temp = []
        for f in fecha:
            try:
                fecha_temp.append(datetime.strptime(f, '%Y%j').strftime('%Y%j'))
            except ValueError:
                try:
                    fecha_temp.append(datetime.strptime(f, '%d-%m-%Y').strftime('%Y%j'))
                except ValueError:
                    raise ValueError(f'Formato de fecha incorrecto para {f}')
        fecha = fecha_temp

    if not isinstance(archivo, list) and len(archivo) == 1:
        if len(archivo) % 18 == 0:
            archivo = [archivo[i:i+18] for i in range(0, len(archivo), 18)]
        else:
            raise ValueError('Nombre del archivo(s) incorrecto')
    elif len(archivo) > 1 and not isinstance(archivo, list):
        archivo = list(archivo)

    if not isinstance(codigo_estacion, list) and codigo_estacion:
        codigo_estacion = [codigo_estacion]
    elif isinstance(codigo_estacion, list):
        codigo_estacion_temp = []
        for c in codigo_estacion:
            codigo_estacion_temp.append(c)
        codigo_estacion = codigo_estacion_temp
        
    if not isinstance(canal, str) and canal:
        canal = [str(c) for c in canal]
    elif isinstance(canal, str) and canal:
        canal = [canal[i:i+1] for i in range(0, max(len(canal), 1))]

    if len(args) > 5 and isinstance(args[5], str):
        dir_arc = args[5]
        if not os.path.isdir(dir_arc):
            raise ValueError('Error en el ingreso del directorio')
    elif len(args) < 6 or not args[5]:
        dir_arc = 'C:/reftek/data'
        if not os.path.isdir(dir_arc):
            raise ValueError('No existe el directorio de los datos')
    else:
        raise ValueError('Error en el ingreso del directorio')

    print(f"Directorio de entrada: {dir_arc}")
    dir_data = os.listdir(dir_arc)
    dir_data = [d for d in dir_data if os.path.isdir(os.path.join(dir_arc, d)) and d not in ('.', '..')]

    if fecha:
        print(fecha)
        dir_data = [d for d in dir_data if d in fecha]
        print(f"Datos encontrados para las fechas: {', '.join(fecha)}")
    print(f"Directorios de datos encontrados: {', '.join(dir_data)}")
    print(f"Estaciones: {codigo_estacion}")

    if len(args) > 7 and isinstance(args[7], str):
        dir_archivos_salida = os.path.join(args[7], 'DATOS_PROCESADOS')
        if not os.path.isdir(dir_archivos_salida):
            os.makedirs(dir_archivos_salida)
    elif len(args) < 8 or not args[7]:
        dir_salida_por_defecto = os.getcwd()
        dir_archivos_salida = os.path.join(dir_salida_por_defecto, 'OUTPUT', 'DATOS_PROCESADOS')

    print(f"Directorio de salida: {dir_archivos_salida}")

    estaciones_dr_c = [[] for _ in range(len(dir_data))]

    # elementos de discretisación de los archivos
    for h in range(len(dir_data)):
        dir_in = [f for f in os.listdir(os.path.join(dir_arc, dir_data[h])) if f not in ('.', '..')]
        # nombre de los directorios
        if codigo_estacion:
            dir_in = [d for d in dir_in if d in codigo_estacion]
        estaciones_dr_c[h] = dir_in
        for l in range(len(dir_in)):
            dir_canal = [f for f in os.listdir(os.path.join(dir_arc, dir_data[h], dir_in[l])) if f not in ('.', '..')]
            if canal and any(c in dir_canal for c in canal):
                dir_canal = [d for d in dir_canal if d in canal]
            elif canal and any(c not in dir_canal for c in canal):
                raise ValueError('El canal introducido no es correcto')

            dir_salida = os.path.join(dir_archivos_salida, dir_in[l])
            dir_salida_temp = os.path.join(dir_salida, datetime.strptime(dir_data[h], '%Y%j').strftime('%d-%m-%Y'))
            if not os.path.isdir(dir_salida_temp):
                os.makedirs(dir_salida_temp)

            dr_c = [[] for _ in range(len(dir_canal))]
            for k in range(len(dir_canal)):
                dir_archivos = [f for f in os.listdir(os.path.join(dir_arc, dir_data[h], dir_in[l], dir_canal[k])) if f not in ('.', '..')]
                if len([d for d in dir_archivos if os.path.isdir(os.path.join(dir_arc, dir_data[h], dir_in[l], dir_canal[k], d))]) == 2:
                    dr_c[k] = []
                    break
                try:
                    dr_c[k] = dir_archivos
                except Exception:
                    print('CELDA VACIA')
                dir_dat_fch = [d for d in dir_data]
            print("Directorio del canal: " + str(dir_canal))

            if len(args) > 6 and isinstance(args[6], str):
                dir_programas = args[6]
                if (not os.path.isdir(dir_programas) or not os.path.exists(dir_programas)) and len(args) > 8 and args[8]:
                    raise ValueError('Error en el ingreso del directorio de programas')
            elif len(args) < 7 or not args[6]:
                dir_programas = 'C:/reftek/bin'
                if not os.path.exists(dir_programas):
                    raise ValueError('No existe el directorio de los datos')
            else:
                raise ValueError('Error en el ingreso del directorio')
            dim_dr_c = [len(dr_c), len(dr_c[0]), len(dr_c[0][0])] if dr_c else [0, 0, 0]

            for fecha in range(dim_dr_c[2]):
                for estacion in range(dim_dr_c[1]):
                    for canales in range(dim_dr_c[0]):
                        if dr_c[canales][estacion][fecha]:
                            temp_archs = [d.name for d in dr_c[canales][estacion][fecha]]
                            if archivo:
                                temp_archs = [t for t in temp_archs if t in archivo]
                                print(f"Archivos encontrados para la fecha {dir_dat_fch[fecha]}, estación {estaciones_dr_c[fecha][estacion]}, canal {canales}: {', '.join(temp_archs)}")

                            dir_salida = os.path.join(dir_archivos_salida, estaciones_dr_c[fecha][estacion])
                            crp_temp = dr_c[canales][estacion][fecha][0].parent
                            for l in range(len(temp_archs)):
                                if len(args) > 8 and args[8]:  # usar programa reftek
                                    linea_cd_stm = f"{os.path.join(dir_programas, programa)} {os.path.join(crp_temp, temp_archs[l])} {os.path.join(dir_salida, datetime.strptime(dir_dat_fch[fecha], '%Y%j').strftime('%d-%m-%Y'))}"
                                    subprocess.run(linea_cd_stm, shell=True)
                                else:
                                    descomprimir_crudos(os.path.join(crp_temp, temp_archs[l]), os.path.join(dir_salida, datetime.strptime(dir_dat_fch[fecha], '%Y%j').strftime('%d-%m-%Y')))  # type: ignore
                            if canales == dim_dr_c[0] - 1:
                                print('Se han procesado los datos crudos')
                        else:
                            pass

            print('Ha finalizado la transformación de los archivos')
            import winsound
            winsound.Beep(440, 500)

In [ ]:
# Ejemplo de uso 1
proc_cru('pas2asc', 'D884', '2021044', '1', '050000000_0036EE80', [], [], 'C:\\Users\\Usuario\\Desktop\\DATA_ESTACION')

# Ejemplo de uso 2
proc_cru('pas2asc', ['D884', 'D8A4'], ['2021044', '2021049'], ['1', '2'], ['050000000_0036EE80', '060000000_0036EE80'], [], [], 'C:\\Users\\Usuario\\Desktop\\DATA_ESTACION')

# Ejemplo de uso 3
proc_cru('pas2asc', 'D884', '2021044', '1', [], 'C:\\reftek\\data', 'C:\\reftek\\bin')



In [105]:
proc_cru('pas2asc', 'D884', '2019258', '1', [], 'C:\Practicas\datos', 'C:\Practicas\programas_r\bin')


Directorio de entrada: C:\Practicas\datos
['2019258']
Datos encontrados para las fechas: 2019258
Directorios de datos encontrados: 2019258
Estaciones: ['D884']
Directorio de salida: C:\Practicas\OUTPUT\DATOS_PROCESADOS
Directorio del canal: ['1']


AttributeError: 'str' object has no attribute 'name'